In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy.stats as stats
import scipy.special
#graphing
import matplotlib.pyplot as plt
#stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from statsmodels.distributions.empirical_distribution import ECDF

#import testing
import sys
sys.path.append("../")
import vuong_tests2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
num_true = 10
num_bidders1 = 10
num_bidders2 = 2
mu = 3
sigma = 2

def gen_data(values=stats.logistic(loc=mu, scale=sigma)):
    nobs = 500
    
    #parameters
    
    
    min_bids =num_true
    max_bids =num_true
    
    bid_types = range(min_bids,max_bids+1)
    prob_type = [1/len(bid_types)]*len(bid_types)
    bidders = np.random.choice(bid_types, nobs, p=prob_type)
    bidders = np.sort(bidders)
    bids = []
    for i in bid_types:
        #count number of obs
        num_i = sum(i == bidders)
        bids_i = values.rvs(size=(num_i,i))
        bids_i =  np.sort(bids_i, axis=1)
        bids_i =  bids_i[:,-2]
        bids = np.concatenate((bids, bids_i))
    
    #draw bids
    return bids,bidders,nobs


values_distr =stats.logistic(loc=mu, scale=sigma)
bids,i,nobs = gen_data(values_distr)
print(bids.shape)
print(bids[i == num_true].mean())

(500,)
6.407482653507039


In [6]:
class Auction(GenericLikelihoodModel):
    
    def __init__(self, *args, values_distr=stats.logistic, **kwargs):
        super(Auction,self).__init__(*args,**kwargs)
        self.values_distr = values_distr
        
    
    def loglikeobs(self, params):
        bids = self.endog
        i = self.exog[:,0]
        cdf = self.values_distr.cdf(bids,loc=params[0],scale=max(params[1],1e-5))
        pdf = self.values_distr.pdf(bids,loc=params[0],scale=max(params[1],1e-5))
        factorial = scipy.special.factorial(i)/scipy.special.factorial(i-2)  
        
        order_cdf = factorial*pdf*cdf**(i-2)*(1-cdf) #np.log(i) + np.log(cdf) + (i-1)*np.log((1-cdf)) #second highest order statistic
        return np.log(order_cdf)

np.random.seed()
yn,xn,nobs = gen_data(stats.logistic(loc=mu, scale=sigma))
model = Auction(yn,xn)
model_fit = model.fit(start_params=[mu,sigma],disp=False)
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               Auction Results                                
==============================================================================
Dep. Variable:                      y   Log-Likelihood:                -982.00
Model:                        Auction   AIC:                             1966.
Method:            Maximum Likelihood   BIC:                             1970.
Date:                Wed, 17 Mar 2021                                         
Time:                        18:38:17                                         
No. Observations:                 500                                         
Df Residuals:                     499                                         
Df Model:                           0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.8835      0.117     24.618      0.000       2.654       3.113
par0           2.0241      0.069     29.501      0.000       1.890       2.159
==============================================================================
"""

In [7]:
def compute_llr(yn,xn):
    
    #fit normal values
    model1 = Auction(yn,xn*num_bidders1/num_true)
    model1_fit = model1.fit(start_params=[mu,sigma],disp=False)
    ll1 = model1.loglikeobs(model1_fit.params)
    
    #fit logistic values
    model2 = Auction(yn,xn*num_bidders2/num_true)
    model2_fit = model2.fit(start_params=[mu,sigma],disp=False)
    ll2 = model2.loglikeobs(model2_fit.params)
    
    llr = ll1.sum() - ll2.sum()
    omega2 = (ll1- ll2).var()
    return llr,np.sqrt(omega2)

yn,xn,nobs = gen_data()
print(compute_llr(yn,xn))

(18.007504031463895, 0.30927797082299563)


In [8]:
def setup_shi(yn,xn):
    model1 = Auction(yn,xn*num_bidders1/num_true)
    model1_fit = model1.fit(start_params=[mu,sigma],disp=False)
    ll1 = model1.loglikeobs(model1_fit.params)
    grad1 =  model1.score_obs(model1_fit.params)    
    hess1 = model1.hessian(model1_fit.params)
    
    #fit logistic values
    model2 = Auction(yn,xn*num_bidders2/num_true)
    model2_fit = model2.fit(start_params=[mu,sigma],disp=False)
    ll2 = model2.loglikeobs(model2_fit.params)
    grad2 =  model2.score_obs(model2_fit.params)    
    hess2 = model2.hessian(model2_fit.params)
    
    return ll1,grad1,hess1,ll2,2, grad2,hess2,2

yn,xn,nobs = gen_data()
ll1,grad1,hess1,ll2,k1, grad2,hess2,k2 = setup_shi(yn,xn)

In [9]:
reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0.03 0.97 0.  ], boot1: [0.03 0.97 0.  ], boot2: [0.03 0.97 0.  ], llr:28.137266024977134, std: 7.639557300494256, omega:0.29785994459612336
[0.09 0.91 0.  ]


In [10]:
num_true = 7
num_bidders1 = 10
num_bidders2 = 2
mu = 3
sigma = 2

def gen_data(values=stats.logistic(loc=mu, scale=sigma)):
    nobs = 500
    
    #parameters
    
    
    min_bids =num_true
    max_bids =num_true
    
    bid_types = range(min_bids,max_bids+1)
    prob_type = [1/len(bid_types)]*len(bid_types)
    bidders = np.random.choice(bid_types, nobs, p=prob_type)
    bidders = np.sort(bidders)
    bids = []
    for i in bid_types:
        #count number of obs
        num_i = sum(i == bidders)
        bids_i = values.rvs(size=(num_i,i))
        bids_i =  np.sort(bids_i, axis=1)
        bids_i =  bids_i[:,-2]
        bids = np.concatenate((bids, bids_i))
    
    #draw bids
    return bids,bidders,nobs


In [11]:
reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0.09 0.91 0.  ], boot1: [0.09 0.91 0.  ], boot2: [0.09 0.91 0.  ], llr:22.46973945794944, std: 6.932846981399138, omega:0.3051705933834835
[0.21 0.79 0.  ]


In [12]:
num_true = 4
num_bidders1 = 10
num_bidders2 = 2
mu = 3
sigma = 2

def gen_data(values=stats.logistic(loc=mu, scale=sigma)):
    nobs = 500
    
    #parameters
    
    
    min_bids =num_true
    max_bids =num_true
    
    bid_types = range(min_bids,max_bids+1)
    prob_type = [1/len(bid_types)]*len(bid_types)
    bidders = np.random.choice(bid_types, nobs, p=prob_type)
    bidders = np.sort(bidders)
    bids = []
    for i in bid_types:
        #count number of obs
        num_i = sum(i == bidders)
        bids_i = values.rvs(size=(num_i,i))
        bids_i =  np.sort(bids_i, axis=1)
        bids_i =  bids_i[:,-2]
        bids = np.concatenate((bids, bids_i))
    
    #draw bids
    return bids,bidders,nobs


In [13]:
reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0.85 0.15 0.  ], boot1: [0.85 0.14 0.01], boot2: [0.85 0.14 0.01], llr:6.073804897687177, std: 8.362307923355997, omega:0.35089125838512303
[0.91 0.09 0.  ]
